In [ ]:
# Import required libraries
import heapq
from collections import deque
import time
import copy

class PuzzleState:
    def __init__(self, board, parent=None, move=None, depth=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.depth = depth
        self.size = len(board)

    def __eq__(self, other):
        return self.board == other.board

    def __hash__(self):
        return hash(str(self.board))

    def __lt__(self, other):
        return False  # For heapq when f-values are equal

    def get_blank_position(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    return i, j
        return None

    def get_neighbors(self):
        neighbors = []
        blank_row, blank_col = self.get_blank_position()

        # Possible moves: up, down, left, right
        moves = [(-1, 0, 'UP'), (1, 0, 'DOWN'), (0, -1, 'LEFT'), (0, 1, 'RIGHT')]

        for dr, dc, move_name in moves:
            new_row, new_col = blank_row + dr, blank_col + dc

            if 0 <= new_row < self.size and 0 <= new_col < self.size:
                new_board = copy.deepcopy(self.board)
                # Swap blank with adjacent tile
                new_board[blank_row][blank_col] = new_board[new_row][new_col]
                new_board[new_row][new_col] = 0

                neighbor = PuzzleState(new_board, self, move_name, self.depth + 1)
                neighbors.append(neighbor)

        return neighbors

    def manhattan_distance(self, goal_state):
        distance = 0
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] != 0:
                    value = self.board[i][j]
                    # Find where this value should be in goal state
                    for gi in range(self.size):
                        for gj in range(self.size):
                            if goal_state.board[gi][gj] == value:
                                distance += abs(i - gi) + abs(j - gj)
                                break
        return distance

    def display(self):
        for row in self.board:
            print([x if x != 0 else ' ' for x in row])
        print()

print("✅ PuzzleState class implemented successfully")


In [ ]:
# Algorithm Implementations

def get_solution_path(state):
    path = []
    current = state
    while current.parent is not None:
        path.append(current.move)
        current = current.parent
    return path[::-1]

def breadth_first_search(initial_state, goal_state):
    """BFS: Guarantees optimal solution"""
    if initial_state == goal_state:
        return [], 1, 0, initial_state
    
    queue = deque([initial_state])
    visited = {initial_state}
    nodes_expanded = 0
    
    while queue:
        current_state = queue.popleft()
        nodes_expanded += 1
        
        for neighbor in current_state.get_neighbors():
            if neighbor == goal_state:
                path = get_solution_path(neighbor)
                return path, nodes_expanded, len(path), neighbor
            
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    
    return None, nodes_expanded, 0, None

def depth_first_search(initial_state, goal_state, max_depth=50):
    """DFS: Memory efficient but may not find optimal solution"""
    if initial_state == goal_state:
        return [], 1, 0, initial_state
    
    stack = [initial_state]
    visited = set()
    nodes_expanded = 0
    
    while stack:
        current_state = stack.pop()
        
        if current_state in visited or current_state.depth > max_depth:
            continue
            
        visited.add(current_state)
        nodes_expanded += 1
        
        if current_state == goal_state:
            path = get_solution_path(current_state)
            return path, nodes_expanded, len(path), current_state
        
        for neighbor in current_state.get_neighbors():
            if neighbor not in visited and neighbor.depth <= max_depth:
                stack.append(neighbor)
    
    return None, nodes_expanded, 0, None

def a_star_search(initial_state, goal_state):
    """A*: Optimal and efficient with Manhattan distance heuristic"""
    if initial_state == goal_state:
        return [], 1, 0, initial_state
    
    open_list = []
    heapq.heappush(open_list, (0, 0, initial_state))  # (f, g, state)
    visited = set()
    nodes_expanded = 0
    
    while open_list:
        f, g, current_state = heapq.heappop(open_list)
        
        if current_state in visited:
            continue
            
        visited.add(current_state)
        nodes_expanded += 1
        
        if current_state == goal_state:
            path = get_solution_path(current_state)
            return path, nodes_expanded, len(path), current_state
        
        for neighbor in current_state.get_neighbors():
            if neighbor not in visited:
                g_new = current_state.depth + 1
                h = neighbor.manhattan_distance(goal_state)
                f_new = g_new + h
                heapq.heappush(open_list, (f_new, g_new, neighbor))
    
    return None, nodes_expanded, 0, None

def greedy_best_first_search(initial_state, goal_state):
    """Greedy: Fast but may find suboptimal solutions"""
    if initial_state == goal_state:
        return [], 1, 0, initial_state
    
    open_list = []
    heapq.heappush(open_list, (0, initial_state))  # (h, state)
    visited = set()
    nodes_expanded = 0
    
    while open_list:
        h, current_state = heapq.heappop(open_list)
        
        if current_state in visited:
            continue
            
        visited.add(current_state)
        nodes_expanded += 1
        
        if current_state == goal_state:
            path = get_solution_path(current_state)
            return path, nodes_expanded, len(path), current_state
        
        for neighbor in current_state.get_neighbors():
            if neighbor not in visited:
                h_new = neighbor.manhattan_distance(goal_state)
                heapq.heappush(open_list, (h_new, neighbor))
    
    return None, nodes_expanded, 0, None

print("✅ All 4 search algorithms implemented successfully")


In [ ]:
# Create test puzzle - Nightmare difficulty (Ultimate Challenge)
def create_test_puzzle():
    # This is one of the most challenging 8-puzzle configurations possible
    # Designed to test algorithms at their absolute limits
    initial_board = [
        [8, 7, 6],
        [0, 4, 1],
        [2, 5, 3]
    ]

    goal_board = [
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 0]
    ]

    return PuzzleState(initial_board), PuzzleState(goal_board)

initial_state, goal_state = create_test_puzzle()

print("Initial State:")
initial_state.display()
print("Goal State:")
goal_state.display()


In [ ]:
# Run comparative analysis of all 4 algorithms
algorithms = [
    ("Breadth-First Search", breadth_first_search),
    ("Depth-First Search", depth_first_search),
    ("A* Search", a_star_search),
    ("Greedy Best-First Search", greedy_best_first_search)
]

results = []

print("🔬 EXPERIMENTAL RESULTS - ALL 4 SEARCH ALGORITHMS")
print("=" * 80)

for name, algorithm in algorithms:
    print(f"\n{name}:")
    start_time = time.time()
    
    # Handle DFS with higher depth limit for this challenging puzzle
    if name == "Depth-First Search":
        result = algorithm(initial_state, goal_state, max_depth=60)
    else:
        result = algorithm(initial_state, goal_state)
    
    path, nodes_expanded, path_length, solution_state = result
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    if path is not None:
        print(f"✅ Solution found!")
        print(f"Path length: {path_length} steps")
        print(f"Nodes expanded: {nodes_expanded:,}")
        print(f"Time taken: {execution_time:.4f} seconds")
        
        results.append({
            'algorithm': name,
            'solved': True,
            'steps': path_length,
            'nodes': nodes_expanded,
            'time': execution_time,
            'efficiency': nodes_expanded / path_length if path_length > 0 else 0
        })
    else:
        print(f"❌ No solution found within limits")
        print(f"Nodes explored: {nodes_expanded:,}")
        print(f"Time taken: {execution_time:.4f} seconds")
        
        results.append({
            'algorithm': name,
            'solved': False,
            'steps': 0,
            'nodes': nodes_expanded,
            'time': execution_time,
            'efficiency': 0
        })

# Summary table
print("\n" + "=" * 80)
print("📊 COMPREHENSIVE PERFORMANCE SUMMARY")
print("=" * 80)
print(f"{'Algorithm':<25} {'Status':<10} {'Steps':<8} {'Nodes':<12} {'Time (s)':<10} {'Efficiency':<10}")
print("-" * 80)

for result in results:
    status = "✅ Solved" if result['solved'] else "❌ Failed"
    steps = str(result['steps']) if result['solved'] else "N/A"
    efficiency = f"{result['efficiency']:.1f}" if result['solved'] else "N/A"
    
    print(f"{result['algorithm']:<25} {status:<10} {steps:<8} {result['nodes']:<12,} {result['time']:<10.4f} {efficiency:<10}")

# Analysis of results
solved_results = [r for r in results if r['solved']]
if solved_results:
    print(f"\n🏆 ALGORITHM PERFORMANCE ANALYSIS:")
    best_steps = min(solved_results, key=lambda x: x['steps'])
    best_time = min(solved_results, key=lambda x: x['time'])
    best_efficiency = min(solved_results, key=lambda x: x['nodes'])
    
    print(f"  • Optimal Solution: {best_steps['algorithm']} ({best_steps['steps']} steps)")
    print(f"  • Fastest Execution: {best_time['algorithm']} ({best_time['time']:.4f}s)")
    print(f"  • Most Efficient: {best_efficiency['algorithm']} ({best_efficiency['nodes']:,} nodes)")


In [ ]:
# Step-by-step visualization helper functions
def get_solution_states(state):
    """Get all states in the solution path including initial state"""
    states = []
    current = state
    while current is not None:
        states.append(current)
        current = current.parent
    return states[::-1]

def visualize_solution_compact(initial_state, solution_state, algorithm_name):
    """Show compact side-by-side solution visualization"""
    if solution_state is None:
        print(f"❌ {algorithm_name}: No solution to visualize")
        return
    
    states = get_solution_states(solution_state)
    moves = get_solution_path(solution_state)
    
    print(f"\n🎬 {algorithm_name} - Step-by-Step Solution ({len(moves)} steps):")
    print("=" * 90)
    
    # Display states in groups of 3 for better readability
    for start_idx in range(0, len(states), 3):
        end_idx = min(start_idx + 3, len(states))
        
        # Print step headers
        step_headers = []
        for idx in range(start_idx, end_idx):
            if idx == 0:
                step_headers.append(f"Step {idx}: Initial".center(20))
            else:
                step_headers.append(f"Step {idx}: {moves[idx-1]}".center(20))
        
        print("  ".join(step_headers))
        
        # Print board rows side by side
        for row in range(3):  # 3x3 puzzle
            row_displays = []
            for idx in range(start_idx, end_idx):
                board_row = str(states[idx].board[row]).replace('0', ' ')
                row_displays.append(board_row.center(20))
            print("  ".join(row_displays))
        
        print()  # Empty line between groups
    
    print("=" * 90)

# Visualization of solution paths for all successful algorithms
print("🎯 SOLUTION PATH VISUALIZATION FOR ALL ALGORITHMS")
print("=" * 90)

# Test all algorithms and collect their solution paths
algorithm_solutions = []

for name, algorithm in algorithms:
    print(f"\n{name}:")
    print("-" * 70)
    
    # Handle DFS with higher depth limit for this challenging puzzle
    if name == "Depth-First Search":
        result = algorithm(initial_state, goal_state, max_depth=60)
    else:
        result = algorithm(initial_state, goal_state)
    
    path, nodes_expanded, path_length, solution_state = result
    
    if path is not None:
        print(f"✅ Solution found in {path_length} steps")
        print(f"Nodes expanded: {nodes_expanded:,}")
        
        # Show complete solution path
        print(f"\n📋 Complete solution path ({path_length} steps):")
        states = get_solution_states(solution_state)
        
        # Display all states in groups of 3
        for start_idx in range(0, len(states), 3):
            end_idx = min(start_idx + 3, len(states))
            
            # Print step headers
            step_headers = []
            for idx in range(start_idx, end_idx):
                if idx == 0:
                    step_headers.append(f"Step {idx}: Initial".center(20))
                elif idx <= len(path):
                    step_headers.append(f"Step {idx}: {path[idx-1]}".center(20))
            
            print("  ".join(step_headers))
            
            # Print board rows side by side
            for row in range(3):
                row_displays = []
                for idx in range(start_idx, end_idx):
                    if idx < len(states):
                        board_row = str(states[idx].board[row]).replace('0', ' ')
                        row_displays.append(board_row.center(20))
                print("  ".join(row_displays))
            
            print()
        
        algorithm_solutions.append({
            'name': name,
            'path': path,
            'length': path_length,
            'nodes': nodes_expanded,
            'solution_state': solution_state
        })
    else:
        print(f"❌ No solution found within constraints")
        print(f"Nodes explored: {nodes_expanded:,}")

# Store results for potential future analysis
pass
